<a href="https://colab.research.google.com/github/Hidestament/AtCoder/blob/main/ABC/ABC185.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A - ABC Preparation

In [1]:
a = list(map(int, input().split()))
print(min(a))

5 3 7 11
3


# B - Smartphone Addiction

## そのままシミュレーション（AC）

In [ ]:
n, m, t = map(int, input().split())
now_time = 0
now_bat = n
for i in range(m):
  a,b = map(int, input().split())  
  now_bat -= (a - now_time)
  if now_bat <= 0:
    exit(print("No"))
  now_bat = min(n, now_bat+(b-a))
  now_time = b

if now_bat - (t - b) > 0:
  print("Yes")
else:
  print("No")

## きれいに修正

In [84]:
n, m, t = map(int, input().split())
td = 0
battery = n
for i in range(m):
  a,b = map(int, input().split())
  battery -= (a - td)
  if battery <= 0:
    exit(print("No"))
  battery = min(n, battery+(b - a))
  td = b
print("Yes" if battery - t + b > 0 else "No")

10 2 20
9 11
13 17


# C - Duodecim Ferra
切り口の個数$L-1$から, 11個を選ぶ組み合わせ $_n C _r$を計算すれば良い.

解答の参考 : 
https://drken1215.hatenablog.com/entry/2020/12/14/015500

Pythonにおける組み合わせの参考：
https://qiita.com/derodero24/items/91b6468e66923a87f39f

## Scipyを使う（PythonでAC）
`scipy.special.comb(n,r)` で, combinationsを計算できる.

第三引数 `exact=True` にするとちゃんと計算してint で返ってくる.

In [96]:
from scipy.special import comb
l = int(input())
ans = comb(l-1, 11, exact=True)
print(ans)

17
4368


## itertoolsを使う（PythonでTLE）
いちいちリスト作らないといけない

In [90]:
import itertools
l = int(input())
l_list = [i for i in range(l-1)]
l_list = list(itertools.combinations(l_list,11))
print(len(l_list))

17
4368


## mathを使う（PythonでAC）
$$
_n C _r = \frac{n!}{r! * (n - r)!}
$$
を計算する.

階上計算は, `math`の`factorial()`を使う

計算誤差の小数点が気になるときは, // でやったりすると良さそう.

In [95]:
import math
l = int(input())
ans = math.factorial(l-1) // math.factorial(11) // math.factorial(l-12)
print(int(ans))

17
4368


## 再帰で自分で計算（PythonでAC）
$_n C _r = _{n-1} C _r + _{n-1} C _{r-1}$ を利用して再帰的に自分で計算する

In [99]:
nCr = {}
def cmb(n, r):
  if r == 0 or r == n:
    return 1
  if r == 1:
    return n
  if (n,r) in nCr:
    return nCr[(n,r)]
  nCr[(n,r)] = cmb(n-1,r) + cmb(n-1, r-1)
  return nCr[(n,r)]

17
4368


# D - Stamp
考え方.

1. 白マスの連続区間長を全部計算する
2. 連続区間長の中で最小のモノをスタンプ長kとする
3. 白マス連続区間長の各要素をkで割って, 天井値(ceil)の総和が答え

ただし, 全部青マスの場合と青マスが存在しない場合には別の処理をする.


## TLEになってしまった


In [65]:
n, m = map(int, input().split())
a = sorted(list(map(int, input().split())))

13 3
13 3 9


In [70]:
m0 = 0
k = 10000
mas = [i for i in range(n)]
ans_list = []
for i in a:
  temp = mas[m0:i-1]
  if temp != []:
    ans_list.append(temp)
  m0 = i

In [74]:
k = len(min(ans_list, key=lambda x:len(x)))

In [79]:
import math
ans = 0
for i in ans_list:
  ans += math.ceil(len(i) / k )
print(ans)

In [81]:
import math
n, m = map(int, input().split())
a = sorted(list(map(int, input().split())))

if m == 0:
  print(1)
elif m == n:
  print(0)
else:
  m0 = 0
  ans_list = []
  mas = [i for i in range(n)]
  for i in a:
    temp = mas[m0:i-1]
    if temp != []:
      ans_list.append(temp)
    m0 = i
  print(ans_list)
  k = len(min(ans_list, key=lambda x:len(x)))
  ans = 0
  for i in ans_list:
    ans += math.ceil(len(i)/k)
  print(ans)

13 3
13 3 9
[[0, 1], [3, 4, 5, 6, 7], [9, 10, 11]]
6


## 模範解答（PythonでAC）
m個の青マスによって, (m+1)個の区間に分けられるので, 予め白マスが連続する区間の長さのリストbを作成してから計算している.

また, Aの入力値を全て-1して, リストのインデックスと対応するようにしている.

In [113]:
n,m = map(int, input().split())
a = sorted(list(map(lambda x:int(x)-1, input().split())))

5 5
5 2 1 4 3


In [114]:
if m == 0:
  ans = 1
else:
  b = [0 for _ in range(m+1)]
  b[0] = a[0] - 0

  for i in range(1,m):
    b[i] = a[i] - a[i-1] - 1
  b[m] = (n-1) - a[m-1]

  if all(i == 0 for i in b):
    ans = 0
  else:
    k = min(i for i in b if i != 0)
    ans = sum(-(-i // k) for i in b)

print(ans)

0


## おまけ：天井関数
連続値 x の小数点を切り上げる関数を一般に天井関数（ceil）と言ったりする.

ex.
- ceil(2.2) = 3
- ceil(-3.4) = -3
- ceil(5.0) = 5

In [118]:
import math
print(math.ceil(2.3))
print(math.ceil(-3.4))
print(math.ceil(5.0))

3
-3
5


`//` の演算が, 余り切り捨てであり, 負の数は大小逆になることを利用して以下のようにしてかける

In [121]:
x = 5
k = 2

# x/k の小数点切り上げを計算したい
ans = - (-x//k)
print(ans)

3
